First, let's import the Python bindings, as usual.

In [1]:
import metapy

In [2]:
metapy.__version__ # you will want your version to be >= to this

'0.2.10'

If you would like to, you can inform MeTA to output log data to stderr like so:

In [3]:
metapy.log_to_stderr()

Now, let's download a list of stopwords and a sample dataset to begin exploring MeTA's topic models.

In [4]:
!wget -N https://raw.githubusercontent.com/meta-toolkit/meta/master/data/lemur-stopwords.txt

--2018-05-01 15:41:33--  https://raw.githubusercontent.com/meta-toolkit/meta/master/data/lemur-stopwords.txt
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.184.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.184.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2747 (2.7K) [text/plain]
Saving to: ‘lemur-stopwords.txt’

lemur-stopwords.txt 100%[===================>]   2.68K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2018-05-01 15:41:33 (45.5 MB/s) - ‘lemur-stopwords.txt’ saved [2747/2747]



In [5]:
!wget -N https://meta-toolkit.org/data/2016-01-26/ceeaus.tar.gz
!tar xf ceeaus.tar.gz

--2018-05-01 15:41:35--  https://meta-toolkit.org/data/2016-01-26/ceeaus.tar.gz
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving meta-toolkit.org (meta-toolkit.org)... 50.116.41.177, 2600:3c02::f03c:91ff:feae:b777
Connecting to meta-toolkit.org (meta-toolkit.org)|50.116.41.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1182344 (1.1M) [application/octet-stream]
Saving to: ‘ceeaus.tar.gz’

ceeaus.tar.gz       100%[===================>]   1.13M  2.30MB/s    in 0.5s    

2018-05-01 15:41:36 (2.30 MB/s) - ‘ceeaus.tar.gz’ saved [1182344/1182344]



We will need to index our data to proceed. We eventually want to be able to extract the bag-of-words representation for our individual documents, so we will want a `ForwardIndex` in this case.

In [6]:
fidx = metapy.index.make_forward_index('ceeaus-config.toml')

 > Counting lines in file: [=================================] 100% ETA 00:00:00 
1525207335: [info]     Creating forward index: ceeaus-idx/fwd (/tmp/pip-req-build-m473bt6z/deps/meta/src/index/forward_index.cpp:239)
 > Tokenizing Docs: [========================================] 100% ETA 00:00:00 
 > Merging: [================================================] 100% ETA 00:00:00 
1525207335: [info]     Done creating index: ceeaus-idx/fwd (/tmp/pip-req-build-m473bt6z/deps/meta/src/index/forward_index.cpp:278)


Just like in classification, the feature set used for the topic modeling will be the feature set used at the time of indexing, so if you want to play with a different set of features (like bigram words), you will need to re-index your data.

For now, we've just stuck with the default filter chain for unigram words, so we're operating in the traditional bag-of-words space.

Let's load our documents into memory to run the topic model inference now.

In [7]:
dset = metapy.learn.Dataset(fidx)

 > Loading instances into memory: [==========================] 100% ETA 00:00:00 


Now, let's try to find some topics for this dataset. To do so, we're going to use a generative model called a topic model.

There are many different topic models in the literature, but the most commonly used topic model is Latent Dirichlet Allocation. Here, we propose that there are K topics (represented with a categorical distribution over words) $\phi_k$ from which all of our documents are genereated. These K topics are modeled as being sampled from a Dirichlet distribution with parameter $\vec{\alpha}$. Then, to generate a document $d$, we first sample a distribution over the K topics $\theta_d$ from another Dirichlet distribution with parameter $\vec{\beta}$. Then, for each word in this document, we first sample a topic identifier $z \sim \theta_d$ and then the word by drawing from the topic we selected ($w \sim \phi_z$). Refer to the [Wikipedia article on LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) for more information.

The goal of running inference for an LDA model is to infer the latent variables $\phi_k$ and $\theta_d$ for all of the $K$ topics and $D$ documents, respectively. MeTA provides a number of different inference algorithms for LDA, as each one entails a different set of trade-offs (inference in LDA is intractable, so all inference algorithms are approximations; different algorithms entail different approximation guarantees, running times, and required memroy consumption). For now, let's run a Variational Infernce algorithm called CVB0 to find two topics. (In practice you will likely be finding many more topics than just two, but this is a very small toy dataset.)

In [8]:
lda_inf = metapy.topics.LDACollapsedVB(dset, num_topics=2, alpha=1.0, beta=0.01)
lda_inf.run(num_iters=1000)

Initialization: [============================================] 100% ETA 00:00:00 
Iteration 1 maximum change in gamma: 1.78732                                     
Iteration 2 maximum change in gamma: 0.397587                                    
Iteration 3 maximum change in gamma: 0.585143                                    
Iteration 4 maximum change in gamma: 0.639287                                    
Iteration 5 maximum change in gamma: 0.543676                                    
Iteration 6 maximum change in gamma: 1.04887                                     
Iteration 7 maximum change in gamma: 1.26393                                     
Iteration 8 maximum change in gamma: 1.43444                                     
Iteration 9 maximum change in gamma: 1.36808                                     
Iteration 10 maximum change in gamma: 1.39221                                    
Iteration 11 maximum change in gamma: 1.3227                                     
Iteration 12 max

The above ran the CVB0 algorithm for 1000 iterations, or until an algorithm-specific convergence criterion was met. Now let's save the current estimate for our topics and topic proportions.

In [9]:
lda_inf.save('lda-cvb0')

We can interrogate the topic inference results by using the `TopicModel` query class. Let's load our inference results back in.

In [10]:
model = metapy.topics.TopicModel('lda-cvb0')

 > Loading topic term probabilities: [=======================] 100% ETA 00:00:00 
 > Loading document topic probabilities: [===================] 100% ETA 00:00:00 


Now, let's have a look at our topics. A typical way of doing this is to print the top $k$ words in each topic, so let's do that.

In [11]:
model.top_k(tid=0)

[(3341, 0.1311039703325231),
 (3045, 0.05434932034406297),
 (2677, 0.036780095760011296),
 (3346, 0.033492639884972024),
 (281, 0.022530673690313033),
 (3729, 0.015620482424303144),
 (1953, 0.012780918673797484),
 (707, 0.012635069663149857),
 (592, 0.011987183284170312),
 (2448, 0.01131774038055637)]

The models operate on term ids instead of raw text strings, so let's convert this to a human readable format by using the vocabulary contained in our `ForwardIndex` to map the term ids to strings.

In [12]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=0)]

[('smoke', 0.1311039703325231),
 ('restaur', 0.05434932034406297),
 ('peopl', 0.036780095760011296),
 ('smoker', 0.033492639884972024),
 ('ban', 0.022530673690313033),
 ('think', 0.015620482424303144),
 ('japan', 0.012780918673797484),
 ('complet', 0.012635069663149857),
 ('cigarett', 0.011987183284170312),
 ('non', 0.01131774038055637)]

In [13]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=1)]

[('time', 0.06705646364996719),
 ('job', 0.056059299650279136),
 ('part', 0.05222306274365633),
 ('student', 0.046429384401537266),
 ('colleg', 0.03488140708901945),
 ('work', 0.029067480910345566),
 ('money', 0.02885021953621179),
 ('think', 0.0223313502030152),
 ('import', 0.02075570151328543),
 ('studi', 0.015483035500804767)]

We can pretty clearly see that this particular dataset was about two major issues: smoking in public and part time jobs for students. This dataset is actually a collection of essays written by students, and there just so happen to be two different topics they can choose from!

The topics are pretty clear in this case, but in some cases it is also useful to score the terms in a topic using some function of the probability of the word in the topic and the probability of the word in the other topics. Intuitively, we might want to select words from each topic that best reflect that topic's content by picking words that both have high probability in that topic **and** have low probability in the other topics. In other words, we want to balance between high probability terms and highly specific terms (this is kind of like a tf-idf weighting). One such scoring function is provided by the toolkit in `BLTermScorer`, which implements a scoring function proposed by Blei and Lafferty.

In [14]:
scorer = metapy.topics.BLTermScorer(model)
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=0, scorer=scorer)]

[('smoke', 0.8741642954704982),
 ('restaur', 0.3174635157613981),
 ('smoker', 0.20060264558827434),
 ('ban', 0.12853037061168004),
 ('cigarett', 0.06557603445386917),
 ('non', 0.06128422163678793),
 ('complet', 0.06105372135982501),
 ('japan', 0.05846453427778453),
 ('health', 0.05054834419152887),
 ('seat', 0.04533989518585457)]

In [15]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=1, scorer=scorer)]

[('job', 0.34822052999812314),
 ('part', 0.31311071220949405),
 ('student', 0.28328931730015805),
 ('colleg', 0.20808996037214555),
 ('time', 0.17797810952278859),
 ('money', 0.16234681881133195),
 ('work', 0.15585089245616857),
 ('studi', 0.08228292617409116),
 ('learn', 0.06491899302248028),
 ('experi', 0.05494526738519956)]

Here we can see that the uninformative word stem "think" was downweighted from the word list from each topic, since it had relatively high probability in either topic.

We can also see the inferred topic distribution for each document.

In [16]:
model.topic_distribution(0)

<metapy.stats.Multinomial {0: 0.021341, 1: 0.978659}>

It looks like our first document was written by a student who chose the part-time job essay topic...

In [17]:
model.topic_distribution(900)

<metapy.stats.Multinomial {0: 0.978797, 1: 0.021203}>

...whereas this document looks like it was written by a student who chose the public smoking essay topic.

We can also infer topics for a brand new document. First, let's create the document and use the forward index we loaded before to convert it to a feature vector:

In [18]:
doc = metapy.index.Document()
doc.content("I think smoking in public is bad for others' health.")
fvec = fidx.tokenize(doc)

Now, let's load a topic model inferencer that uses the same CVB inference method we used earlier:

In [19]:
inferencer = metapy.topics.CVBInferencer('lda-cvb0.phi.bin', alpha=1.0)

 > Loading topic term probabilities: [=======================] 100% ETA 00:00:00 


Now, let's infer the topic proportions for the new document:

In [20]:
proportions = inferencer.infer(fvec, max_iters=20, convergence=1e-4)
print(proportions)

<metapy.stats.Multinomial {0: 0.814392, 1: 0.185608}>
